In [ ]:
import astropy.units as u
from astroquery.ipac.ned import Ned
from astropy import coordinates
import pandas as pd
import numpy as np


## Defining the function for redshift search

In [ ]:
def radial_search_redshift_in_ned(ra, dec, radius) :
    # ra (array of shape [n]) in hour min sec
    # dec (array of shape [n]) in degree arcmin arcsec
    # radius(scalar float) in arcsec
    
    co = coordinates.SkyCoord(ra=ra, dec=dec, unit=(u.hourangle,u.deg))
    result_table = Ned.query_region(co, radius=radius * u.arcsec, equinox='J2000')
    redshift = result_table['Redshift']
    return float(np.average(redshift))

## Reading the RA-DEC of the sources

In [ ]:
#Reading the ra-dec
source_list = pd.read_csv('spec_info.csv')
source_no = source_list['source_no.']

ra = []
dec = []

for i in range(len(source_no)):
    det_path = source_no[i]
    with open(f'{det_path}/details.txt') as details :
        lines = details.readlines()
        ra.append(lines[1][5:-1])
        dec.append(lines[2][5:-1])


#searching for redshift
radius = float(input('Please enter the search radius(in arcsec) '))
total_source = len(ra)

z = []
for i in range(total_source):
    z.append(radial_search_redshift_in_ned(ra[i], dec[i], radius))

source_list.insert(4, column='redshift', value = z)
source_list.to_csv('spec_info.csv', index = False)
